In [1]:
from torch import nn, optim

from wavenet.audiodata import AudioData, AudioLoader
from wavenet.models_torch import Model

from IPython.display import Audio

%matplotlib inline

x_len = 80
y_len = 1
num_classes = 256
num_blocks = 2
num_layers = 14
num_hidden = 128
kernel_size = 2
learn_rate = 0.001
step_size = 50
gamma = 0.5
num_workers = 1
batch_size = 1

## Create dataset

In [2]:
filelist = ['assets/sine.wav']
dataset = AudioData(filelist, x_len, y_len, store_tracks=True)

Audio(dataset.tracks[0]['audio'], rate=dataset.tracks[0]['sample_rate'])

## Create dataloader

In [3]:
dataloader = AudioLoader(dataset, batch_size=batch_size, 
                         num_workers=num_workers)

## Define training parameters

In [4]:
wave_model = Model(x_len, num_channels=1, num_classes=num_classes, 
                   num_blocks=num_blocks, num_layers=num_layers,
                   num_hidden=num_hidden, kernel_size=kernel_size)

wave_model.criterion = nn.CrossEntropyLoss()
wave_model.optimizer = optim.Adam(wave_model.parameters(), 
                                  lr=learn_rate)
wave_model.scheduler = optim.lr_scheduler.StepLR(wave_model.optimizer, 
                                                 step_size=step_size, 
                                                 gamma=gamma)

## Train model

In [5]:
wave_model.train(dataloader)

Epoch 0 / 24
----------


/home/masonmcgough/anaconda2/envs/PyTorchSIIM/lib/python3.6/site-packages/torch/cuda/__init__.py:116: UserWarning: 
    Found GPU0 Quadro K4000 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same